In [ ]:
# from google.colab import files
# from google.colab import drive
# drive.mount('/drive')

Mounted at /drive


## Loading the data

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import random
random.seed(420)


url = 'https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_test.data'
df_test = pd.read_csv(url, header=None, sep =' ')
df_test = df_test.dropna(how='all', axis=1)

url = 'https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_train.data'
X_train = pd.read_csv(url, header=None, sep =' ')
X_train = X_train.dropna(how='all', axis=1)

url = 'https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_train.labels'
y_train = pd.read_csv(url, header=None)

## Voting classifier

In [ ]:
from sklearn.preprocessing import LabelEncoder

y_train2 = LabelEncoder().fit_transform(y_train)
y_train2

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, ..., 0, 1, 1])

In [ ]:
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

rfc = RandomForestClassifier(random_state=21)
gbc = GradientBoostingClassifier(random_state=21)
xgb = XGBClassifier(random_state=21)

anova = SelectKBest(score_func=f_classif,k=15)
vc = VotingClassifier(estimators=[('rfc', rfc), ('gbc', gbc),  ('xgb', xgb)], voting='soft')

pipe = Pipeline([('anova', anova), ('vc', vc)])

In [ ]:
pipe.get_params(['deep'])

{'memory': None,
 'steps': [('anova', SelectKBest(k=15)),
  ('vc',
   VotingClassifier(estimators=[('rfc', RandomForestClassifier(random_state=21)),
                                ('gbc',
                                 GradientBoostingClassifier(random_state=21)),
                                ('xgb',
                                 XGBClassifier(base_score=None, booster=None,
                                               callbacks=None,
                                               colsample_bylevel=None,
                                               colsample_bynode=None,
                                               colsample_bytree=None, device=None,
                                               early_stopping_rounds=None,
                                               enable_categorical=False,
                                               eval_metric=None,
                                               feature_...
                                               grow_pol

In [ ]:
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV

grid_params_rs = [{
    "vc__gbc__n_estimators": randint(1,500),
    "vc__gbc__learning_rate": uniform(0.01,0.99),
    "vc__gbc__max_leaf_nodes":  randint(2,50),
    "vc__gbc__min_samples_leaf": randint(1,50),
    "vc__gbc__max_depth": randint(1,15),
    'vc__rfc__bootstrap': [True, False],   # replace
    'vc__rfc__max_features': uniform(0.01, 1),   # mtry
    'vc__rfc__min_samples_split': uniform(0.01, 1),  # min.node.size
    'vc__rfc__n_estimators': randint(1, 2001),
    "vc__xgb__n_estimators": randint(1,5000),
    "vc__xgb__learning_rate": uniform(0,10),
    "vc__xgb__max_depth": randint(1,15),
    "vc__xgb__min_child_weight": uniform(0,7)
}]

In [ ]:
bscv = RandomizedSearchCV(pipe, grid_params_rs, random_state=21, scoring="balanced_accuracy", n_iter=50)
bscv.fit(X_train, y_train2)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py", line 346, in fit
    return super().fit(X, transformed_y, sample_weight)
  File "/usr/local

RandomizedSearchCV(estimator=Pipeline(steps=[('anova', SelectKBest(k=15)),
                                             ('vc',
                                              VotingClassifier(estimators=[('rfc',
                                                                            RandomForestClassifier(random_state=21)),
                                                                           ('gbc',
                                                                            GradientBoostingClassifier(random_state=21)),
                                                                           ('xgb',
                                                                            XGBClassifier(base_score=None,
                                                                                          booster=None,
                                                                                          callbacks=None,
                                                                                          colsample_bylevel=None,
                                                                                          colsample_bynode=None,
                                                                                          colsample_bytree=None,
                                                                                          device=Non...
                                         'vc__xgb__max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7860ca4c9d80>,
                                         'vc__xgb__min_child_weight': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7860ca4caa10>,
                                         'vc__xgb__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7860ca4ca080>}],
                   random_state=21, scoring='balanced_accuracy')

In [ ]:
results_bs = pd.DataFrame()
results_bs["params"] = bscv.cv_results_["params"]
results_bs["a_scores"] = bscv.cv_results_["mean_test_score"]

results_bs.to_csv("/drive/My Drive/AutoML/hw2/ensemble/ANOVA_ensemble_results_rs.csv",index=False)

In [ ]:
bscv.best_score_


0.8384999999999998

In [ ]:
ensemble = bscv.best_estimator_
ensemble_proba = pd.DataFrame(ensemble.predict_proba(df_test)[:,1])
np.savetxt('probs_anova_ensemble.txt', ensemble.predict_proba(df_test)[:,1], fmt = '%10.25f')

In [ ]:
np.savetxt('probs_anova_ensemble2.txt', ensemble.predict_proba(df_test)[:,1], fmt = '%10.25f')